# Exemple de profiling

Profiling et fonction *pdf*. Le profiling est utilisé pour mesurer le temps que passe un programme dans chaque fonction.

## Bizarrerie

C'est un exemple qui m'a été envoyé par un étudiant pendant l'été pour montrer que la fonction [pdf](http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.stats.norm.html) est plus lente qu'une réimplémentation simple qui fait à la même chose.

In [ ]:
import time
from scipy.stats import norm
import numpy as np

In [ ]:
debut=time.time()
for i in range(10**3):
    norm(2,3).pdf(4)
fin=time.time()
fin-debut

1.5188484191894531

In [ ]:
def density(x,mean,sigma2):
    return np.exp(-(x-mean)**2/(2*sigma2))/(2*np.pi*sigma2)**0.5

debut=time.time()
for i in range(10**3):
    density(4,2,3)
fin=time.time()
fin-debut

0.008997917175292969

**Que se passe-t-il ?**

Tout d'abord la fonction [pdf](http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.stats.norm.html) comme toute les fonctions des librairies numériques sont optimisées pour le calcul sur des matrices ou des vecteurs et non sur des nombres. Pour la suite, on utilise un profileur.

## Profiler

In [ ]:
import cProfile, io, pstats, os, sys
def doprofile(func, filename, *l):
    pr = cProfile.Profile()
    pr.enable()  # début du profiling
    func(*l)     # appel de la fonction                
    pr.disable() # fin du profiling
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.print_stats()
    rem = os.path.normpath(os.path.join(os.getcwd(), "..", "..", ".."))
    res = s.getvalue().replace(rem, "")
    res = res.replace(sys.base_prefix, "").replace("\\", "/")
    ps.dump_stats(filename)
    return res

In [ ]:
import numpy
x = numpy.ones((10000000, 1))*4
x.shape

(10000000, 1)

In [ ]:
debut=time.time()
y = norm.pdf(x)
fin=time.time()
print(fin-debut, y.shape, y[0])

1.1308369636535645 (10000000, 1) [ 0.00013383]


In [ ]:
r = doprofile(norm.pdf, "pdf.prof", x)
print(r)

         106 function calls in 1.086 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.213    0.213    1.086    1.086 /lib/site-packages/scipy/stats/_distn_infrastructure.py:1619(pdf)
        1    0.011    0.011    0.303    0.303 /lib/site-packages/scipy/stats/_distn_infrastructure.py:520(argsreduce)
        1    0.080    0.080    0.292    0.292 /lib/site-packages/scipy/stats/_distn_infrastructure.py:545(<listcomp>)
        1    0.000    0.000    0.286    0.286 /lib/site-packages/scipy/stats/_continuous_distns.py:129(_pdf)
        1    0.286    0.286    0.286    0.286 /lib/site-packages/scipy/stats/_continuous_distns.py:79(_norm_pdf)
        1    0.000    0.000    0.226    0.226 /lib/site-packages/numpy/lib/function_base.py:1964(place)
        1    0.226    0.226    0.226    0.226 {built-in method numpy.core.multiarray._insert}
        2    0.012    0.006    0.212    0.106 /lib/site-packages/numpy/lib/functio

In [ ]:
def density(x,mean,sigma2):
    return np.exp(-(x-mean)**2/(2*sigma2))/(2*np.pi*sigma2)**0.5

debut=time.time()
y = density(x,0.0,1.0)
fin=time.time()
print(fin-debut, y.shape, y[0])

0.36240410804748535 (10000000, 1) [ 0.00013383]


In [ ]:
r = doprofile(density, "pdf.prof", x, 0, 1)
print(r)

         2 function calls in 0.341 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.341    0.341    0.341    0.341 <ipython-input-15-e74f743d975f>:1(density)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}





Quand on regarde le code de la fonction, on s'aperçoit que la fonction perd du temps dans [argsreduce](https://github.com/scipy/scipy/blob/v0.17.1/scipy/stats/_distn_infrastructure.py#L521). Elle fait aussi d'autres choses comme regarder les valeurs manquantes. En guise de conclusion, lorsqu'une fonction gère trop de cas particuliers (type, valeurs), elle est nécessairement plus lente qu'une fonction qu'on implémente soi-même.